<a href="https://colab.research.google.com/github/yodaboop/Capstone-Project/blob/main/MDM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. INSTALL DEPENDENCIES

!sudo apt-get -y -qq update > /dev/null 2>&1
!sudo apt-get -y -qq install postgresql > /dev/null 2>&1
!sudo service postgresql start

# 2. SETUP DATABASE & USER
import subprocess

# Define the shell command as a Python string
setup_db_cmd = """
sudo -u postgres psql << EOF
CREATE USER toolbox_user WITH PASSWORD 'my-password';
CREATE DATABASE toolbox_db;
GRANT ALL PRIVILEGES ON DATABASE toolbox_db TO toolbox_user;
ALTER DATABASE toolbox_db OWNER TO toolbox_user;
EOF
"""

# Run it using subprocess
subprocess.run(setup_db_cmd, shell=True)

 * Starting PostgreSQL 14 database server
   ...done.


CompletedProcess(args="\nsudo -u postgres psql << EOF\nCREATE USER toolbox_user WITH PASSWORD 'my-password';\nCREATE DATABASE toolbox_db;\nGRANT ALL PRIVILEGES ON DATABASE toolbox_db TO toolbox_user;\nALTER DATABASE toolbox_db OWNER TO toolbox_user;\nEOF\n", returncode=0)

In [ ]:
import subprocess
import os

# Set the PGPASSWORD environment variable for the subprocess
env = os.environ.copy()
env["PGPASSWORD"] = "my-password"

# Define the shell command as a Python string
seed_db_cmd = """
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF

DROP TABLE IF EXISTS devices;

-- 1. DEVICES TABLE
CREATE TABLE devices (
    id SERIAL PRIMARY KEY,
    serial_number VARCHAR(50) UNIQUE NOT NULL,
    assigned_email VARCHAR(100) UNIQUE NOT NULL,
    model VARCHAR(50),
    status VARCHAR(20) DEFAULT 'unprovisioned'
);

-- SEED DATA (Aligned with Identity Agent Users)
-- neo (admin) -> Active device
-- trinity (user) -> Active device
-- smith (agent) -> Unprovisioned (Needs setup)
INSERT INTO devices (serial_number, assigned_email, model, status) VALUES
('SN-NEO-001', 'neo@matrix.com', 'Nebuchadnezzar Mainframe', 'active'),
('SN-TRIN-002', 'trinity@matrix.com', 'Ducati Workstation', 'active'),
('SN-SMITH-666', 'smith@matrix.com', 'Glitch Laptop', 'unprovisioned');

SELECT * FROM devices;
EOF
"""

# Run it using subprocess, passing the environment variables
subprocess.run(seed_db_cmd, shell=True, env=env)

CompletedProcess(args="\npsql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF\n\nDROP TABLE IF EXISTS devices;\n\n-- 1. DEVICES TABLE\nCREATE TABLE devices (\n    id SERIAL PRIMARY KEY,\n    serial_number VARCHAR(50) UNIQUE NOT NULL,\n    assigned_email VARCHAR(100) UNIQUE NOT NULL,\n    model VARCHAR(50),\n    status VARCHAR(20) DEFAULT 'unprovisioned'\n);\n\n-- SEED DATA (Aligned with Identity Agent Users)\n-- neo (admin) -> Active device\n-- trinity (user) -> Active device\n-- smith (agent) -> Unprovisioned (Needs setup)\nINSERT INTO devices (serial_number, assigned_email, model, status) VALUES\n('SN-NEO-001', 'neo@matrix.com', 'Nebuchadnezzar Mainframe', 'active'),\n('SN-TRIN-002', 'trinity@matrix.com', 'Ducati Workstation', 'active'),\n('SN-SMITH-666', 'smith@matrix.com', 'Glitch Laptop', 'unprovisioned');\n\nSELECT * FROM devices;\nEOF\n", returncode=0)

In [ ]:
# ==========================================
# 3. SETUP TOOLBOX BINARY
# ==========================================
import os
version = "0.22.0"
if not os.path.exists("toolbox"):
    !curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox
    !chmod +x toolbox

TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5001

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  158M  100  158M    0     0   117M      0  0:00:01  0:00:01 --:--:--  118M


In [ ]:
# ==========================================
# 4. DEFINE MDM TOOLS (tools.yml)
# ==========================================
tools_file_name = "mdm_tools.yml"

file_content = """
sources:
  mdm-pg-source:
    kind: postgres
    host: 127.0.0.1
    port: 5432
    database: toolbox_db
    user: toolbox_user
    password: my-password

tools:
  get-device-serial:
    kind: postgres-sql
    source: mdm-pg-source
    description: Retrieve the serial number and status for a user's assigned device.
    parameters:
      - name: email
        type: string
        description: The email address of the employee.
    statement: SELECT serial_number, model, status FROM devices WHERE assigned_email = $1;

  setup-device:
    kind: postgres-sql
    source: mdm-pg-source
    description: Provision a device by setting its status to 'active'.
    parameters:
      - name: serial_number
        type: string
        description: The serial number of the device to setup.
    statement: UPDATE devices SET status = 'active' WHERE serial_number = $1;

  remote-wipe-device:
    kind: postgres-sql
    source: mdm-pg-source
    description: Wipe a device remotely by setting status to 'wiping'.
    parameters:
      - name: serial_number
        type: string
        description: The serial number of the device to wipe.
    statement: UPDATE devices SET status = 'wiping' WHERE serial_number = $1;

toolsets:
  mdm-toolset:
    - get-device-serial
    - setup-device
    - remote-wipe-device
"""

with open(tools_file_name, "w") as f:
    f.write(file_content)

In [ ]:
# Install the Toolbox Langchain package
!pip install toolbox-langchain --quiet
!pip install langgraph --quiet

# Install the Langchain llm package
# TODO(developer): replace this with another model if needed
! pip install langchain-google-genai --quiet
# ! pip install langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.8/475.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
import subprocess
import time
import asyncio
from google.colab import userdata
from toolbox_langchain import ToolboxClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
import os # Import os module for os.environ
import google.generativeai as genai # Import the google.generativeai library

# Start Server
log_file = open("mdm_toolbox.log", "w")
process = subprocess.Popen(
    ["./toolbox", "--tools-file", tools_file_name, "--port", str(SERVER_PORT)],
    stdout=log_file,
    stderr=log_file
)
time.sleep(5)

# --- MDM AGENT LOGIC ---
system_prompt = """
You are the MDM Agent. Manage hardware life-cycles.
RULES:
1. Always check 'get_device_serial' before acting.
2. Use 'setup_device' for onboarding.
3. Use 'remote_wipe_device' for offboarding.
"""

# UPDATED QUERIES TO MATCH IDENTITY DATA
queries = [
    # Scenario 1: Smith exists in Identity but his device is 'unprovisioned'.
    # This tests the Onboarding flow.
    "I need to onboard Agent Smith (smith@matrix.com). Please setup his device.",

    # Scenario 2: Trinity is leaving.
    # This tests the Offboarding flow.
    "Trinity (trinity@matrix.com) is leaving the Matrix. Please wipe her workstation.",

    # Scenario 3: Neo needs a status check.
    # This tests simple retrieval.
    "What is the status of Neo's device (neo@matrix.com)?"
]

async def run_mdm_agent():
    api_key = userdata.get('GEMINI_API_KEY')

    global mdm_graph

    if not api_key:
        print("ERROR: GEMINI_API_KEY not found in Colab secrets. Please set it to proceed.")
        return

    os.environ['GEMINI_API_KEY'] = api_key
    genai.configure(api_key=api_key) # Configure genai with the API key

    # Try to list available models to diagnose the NOT_FOUND error
    available_models = []
    try:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                available_models.append(m.name)
        print(f"Available models for generateContent: {available_models}")
    except Exception as e:
        print(f"Error listing models: {e}")
        print("Please ensure your GEMINI_API_KEY is valid and has access to Gemini models.")
        return

    # Use the model requested by the user
    selected_model = "gemini-2.5-flash"

    if selected_model not in available_models:
        print(f"WARNING: Requested model '{selected_model}' not found in available models. Falling back to the first available model.")
        if available_models:
            selected_model = available_models[0]
        else:
            print("ERROR: No suitable Gemini models found for content generation. Please check your API key and model access.")
            return

    print(f"Using model: {selected_model}")
    model = ChatGoogleGenerativeAI(model=selected_model, google_api_key=api_key)

    async with ToolboxClient(f"http://127.0.0.1:{SERVER_PORT}") as client:
        tools = await client.aload_toolset()
        agent = create_react_agent(model, tools, checkpointer=MemorySaver())
        mdm_graph = agent # Moved this line to after 'agent' is created
        config = {"configurable": {"thread_id": "mdm-session-1"}}

        print("\n--- MDM AGENT ONLINE (MATRIX EDITION) ---\n")

        for query in queries:
            print(f"🔹 User: {query}")
            inputs = {"messages": [("user", query)]}
            response = await agent.ainvoke(inputs, config=config)
            print(f"🤖 Agent: {response['messages'][-1].content}\n")
            print("-" * 40)

try:
    await run_mdm_agent()
finally:
    process.terminate()
    print("Server stopped.")

Available models for generateContent: ['models/gemini-2.5-flash', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'models/gemini-flash-latest', 'models/gemini-flash-lite-latest', 'models/gemini-pro-latest', 'models/gemini-2.5-flash-lite', 'models/gemini-2.5-flash-image-preview', 'models/gemini-2.5-flash-image', 'models/gemini-2.5-flash-preview-09-2025', 'models/gemini-2.5-flash-lite-preview-09-2025', 'models/gemi

/tmp/ipython-input-3356901778.py:86: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, tools, checkpointer=MemorySaver())



--- MDM AGENT ONLINE (MATRIX EDITION) ---

🔹 User: I need to onboard Agent Smith (smith@matrix.com). Please setup his device.
🤖 Agent: [{'type': 'text', 'text': 'I can help with that. What is the serial number of the device? If you provide the serial number, I can set up the device for Agent Smith. Alternatively, I can retrieve the serial number using his email address (smith@matrix.com).', 'extras': {'signature': 'CrwJAXLI2nzovelglAjVQCD2n46wCVRMWjswIfdmZ3Y27Yo3Ye0T2IxzIFz9kZrhOMyk5EHmuq6Z2leUoiX3gpwzuKPx114y+3r7wnGgRLcV6L7xgfataJ0gJehTfT/ICabUrPjcgHPc9sDXGtRGtcPsDWLzrr8za+NL9de0+4Jr0atWC4biJuS82fWCgfD/zFmSMZDYbik9ermCsPF2ST0XMBdt04ACjl/+qfbkgdxs0w0yEn92FMlOukhyws2sF1BOCaAtsvDRj3q2gvVh+dxyIAtUPTh0s6qTV0oQL4l85cJ8bSUdmPp6jLHycwAO0NNK71/VqjCrqJzFI1IdJwJLi3w2VJE3bKvvAEEjYND3V+VHjMv7IgGTtVQ+XtQ/CPfV49pM3GsB8ikoJ0RX0i/ktdjsfHbSsaDqqrsYHAJgtCU4OL/nVoVl0braFFimCLRo9DrD5ntWQMBM4ISyXqfE0esxOgDJuQnCgzEBjc1Oy6wWMDLv5zGo4RD/ypg1O6wFflg5H0HkSnIVP5yivZSoO8QLPUr5V35Vv6ixZtERyiij2Fm8S8pqJRu+FjUyB9cg